## Step 1. 데이터 다운로드
- 이미 실습(1) 데이터 다듬기에서 Cloud shell에 심볼릭 링크로 ~/aiffel/lyricist/data를 생성하셨다면
- ~/aiffel/lyricist/data/lyrics에 데이터가 있습니다.

## Step 2. 데이터 읽어오기
- glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요
- glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 할께요!

In [1]:
import os, re 
import numpy as np
import tensorflow as tf

from glob import glob

# glob 를 활용하여 모든 txt 파일을 읽어오기 = txt_file_list

file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'
txt_file_list = glob(file_path)   

print("텍스트 파일 갯수:", len(txt_file_list))
# print(txt_file_list)

# raw_corpus 리스트에 문장 단위로 저장

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.

for txt_file in txt_file_list:
    
    # 파일을 읽기모드로 열고
    # 라인 단위로 끊어서 list 형태로 읽어옵니다.

    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

# 앞에서부터 10라인, 뒤에서부터 10라인만 화면에 출력해 볼까요?
# print(raw_corpus[:9])
# print(raw_corpus[-10:])

print("데이터 크기:", len(raw_corpus))
print("첫 번째 문장:", raw_corpus[0])
print("센텐스 크기:", len(raw_corpus[0]))
print("단어 수:", len(raw_corpus[0].split()))

텍스트 파일 갯수: 49
데이터 크기: 187088
첫 번째 문장: Now I've heard there was a secret chord
센텐스 크기: 39
단어 수: 8


## Step 3. 데이터 정제
- 앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!
- preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

In [2]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


- 추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다
  - 너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠
  - 그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기를 권합니다.

In [3]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    #토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외
    if len(sentence.split()) > 15: continue
        
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
print("정제된 데이터 크기:", len(corpus))


정제된 데이터 크기: 168357


이제 데이터는 완벽하게 준비가 된 것 같네요!

## Step 4. 평가 데이터셋 분리
- tokenize() 함수로 데이터를 Tensor로 변환한 후
- 단어장의 크기는 12,000 이상 으로 설정하세요!

In [4]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2706 ...    0    0    0]
 [   2   34    7 ...   44    3    0]
 ...
 [   2  259  194 ...   12    3    0]
 [   5   22    9 ...   10 1099    3]
 [   2    7   33 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f90881e04c0>


In [5]:
print(tensor.shape)

(168357, 15)


- 단어 사전이 어떻게 구축되었는지 아래와 같이 확인해 봅시다.

In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx], end=" ")
    if idx >= 10: break
print(len(tokenizer.index_word.keys()))

1 : <unk> 2 : <start> 3 : <end> 4 : , 5 : i 6 : the 7 : you 8 : and 9 : a 10 : to 26729


### 생성된 텐서를 소스와 타겟으로 분리  
- 이 과정도 텐서플로우가 제공하는 모듈을 사용할 것이니, 어떻게 사용하는지만 눈여겨 봐둡시다.

- 훈련 데이터와 평가 데이터를 분리하세요!
- sklearn 모듈의 train_test_split() 함수를 사용해 **훈련 데이터**와 **평가 데이터**를 분리하도록 하겠습니다
- 총 데이터의 **20%**를 평가 데이터셋으로 사용해 주세요!

In [7]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 42)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

[   2   50    5   91  307   62   57    9  957 5739    3    0    0    0]
[  50    5   91  307   62   57    9  957 5739    3    0    0    0    0]
Source Train: (134685, 14)
Target Train: (134685, 14)


corpus 내의 첫 번째 문장에 대해 생성된 소스와 타겟 문장을 확인해 보았습니다.
- 예상대로 소스는 2([start])에서 시작해서 3([end]으로 끝난 후 0([pad])로 채워져 있습니다
- 하지만 타겟은 2로 시작하지 않고 소스를 왼쪽으로 한 칸 시프트 한 형태를 가지고 있습니다

### 데이터셋 객체 생성

In [8]:
# BUFFER_SIZE = len(src_input)
# BATCH_SIZE = 256
# steps_per_epoch = len(src_input) // BATCH_SIZE

BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 120001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다

# dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
# dataset = dataset.shuffle(BUFFER_SIZE)
# dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

### 모델 구성하기
- tf.keras.Model을 Subclassing하는 방식
- 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성
- Embedding Layer의 ***embedding_size***=워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
  - 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
  - 그만큼 충분한 데이터가 주어지지 않으면 오히려 혼란만을 야기
  - 이번 실습에서는 256이 적당해 보인다고...
- LSTM 레이어의 hidden state의 차원 수인 ***hidden_size***도 같은 맥락
  - hidden_size는 모델에 얼마나 많은 일꾼을 둘 것인가?로 이해해도 크게 엇나가지 않음
  - 그 일꾼들은 모두 같은 데이터를 보고 각자의 생각을 가지는데
  - 역시 충분한 데이터가 주어지면 올바른 결정을 내리겠지만 그렇지 않으면 배가 산으로 갈 뿐
  - 이번 실습에는 1024가 적당해 보이는군요...

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

#### (for Step 6) 아래 generate_text 함수는 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행하게 합니다.

In [10]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

  
## Step 5-1. 인공지능 만들기
- 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요!
- (Loss는 아래 제시된 Loss 함수를 그대로 사용!)
    - loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
- 그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

In [11]:
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
# model.summary()

- 이제 모델이 학습할 준비가 완료되었습니다. 
- 아래 코드를 실행해 모델을 학습시켜보세요!

In [13]:
# optimizer와 loss등은 차차 배웁니다

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val))

Epoch 1/10
526/526 [==============================] - 96s 172ms/step - loss: 3.6285 - val_loss: 3.2538
Epoch 2/10
526/526 [==============================] - 99s 189ms/step - loss: 3.1459 - val_loss: 3.0635
Epoch 3/10
526/526 [==============================] - 100s 191ms/step - loss: 2.9693 - val_loss: 2.9396
Epoch 4/10
526/526 [==============================] - 100s 191ms/step - loss: 2.8340 - val_loss: 2.8470
Epoch 5/10
526/526 [==============================] - 101s 191ms/step - loss: 2.7180 - val_loss: 2.7785
Epoch 6/10
526/526 [==============================] - 100s 190ms/step - loss: 2.6114 - val_loss: 2.7248
Epoch 7/10
526/526 [==============================] - 101s 191ms/step - loss: 2.5114 - val_loss: 2.6746
Epoch 8/10
526/526 [==============================] - 101s 191ms/step - loss: 2.4155 - val_loss: 2.6341
Epoch 9/10
526/526 [==============================] - 100s 191ms/step - loss: 2.3245 - val_loss: 2.5988
Epoch 10/10
526/526 [==============================] - 101s 191ms/

## Step 6-1. 작문 모델 평가 - 작문 시켜보고 직접 평가

In [14]:
# 작문 모델 평가 - 작문 시켜보고 직접 평가
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

  
## Step 5-2. 인공지능 만들기: hyperparameter 변경

In [15]:
embedding_size = 256*2
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [16]:
# optimizer와 loss등은 차차 배웁니다

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val))

Epoch 1/10
526/526 [==============================] - 109s 200ms/step - loss: 3.5828 - val_loss: 3.2055
Epoch 2/10
526/526 [==============================] - 105s 199ms/step - loss: 3.0911 - val_loss: 3.0076
Epoch 3/10
526/526 [==============================] - 105s 199ms/step - loss: 2.9091 - val_loss: 2.8863
Epoch 4/10
526/526 [==============================] - 105s 199ms/step - loss: 2.7658 - val_loss: 2.7943
Epoch 5/10
526/526 [==============================] - 105s 200ms/step - loss: 2.6397 - val_loss: 2.7219
Epoch 6/10
526/526 [==============================] - 105s 200ms/step - loss: 2.5223 - val_loss: 2.6643
Epoch 7/10
526/526 [==============================] - 105s 199ms/step - loss: 2.4124 - val_loss: 2.6137
Epoch 8/10
526/526 [==============================] - 105s 200ms/step - loss: 2.3068 - val_loss: 2.5736
Epoch 9/10
526/526 [==============================] - 105s 200ms/step - loss: 2.2062 - val_loss: 2.5395
Epoch 10/10
526/526 [==============================] - 105s 200m

## Step 6-2. 작문 모델 평가 - 작문 시켜보고 직접 평가

In [17]:
# 작문 모델 평가 - 작문 시켜보고 직접 평가
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

  
## Step 5-3. 인공지능 만들기: hyperparameter 변경

In [18]:
embedding_size = 256
hidden_size = 1024*2
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [19]:
# optimizer와 loss등은 차차 배웁니다

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val))

Epoch 1/10
526/526 [==============================] - 293s 523ms/step - loss: 3.5300 - val_loss: 3.1063
Epoch 2/10
526/526 [==============================] - 276s 524ms/step - loss: 2.9534 - val_loss: 2.8499
Epoch 3/10
526/526 [==============================] - 276s 524ms/step - loss: 2.6775 - val_loss: 2.6765
Epoch 4/10
526/526 [==============================] - 276s 524ms/step - loss: 2.4150 - val_loss: 2.5512
Epoch 5/10
526/526 [==============================] - 276s 524ms/step - loss: 2.1591 - val_loss: 2.4494
Epoch 6/10
526/526 [==============================] - 276s 525ms/step - loss: 1.9180 - val_loss: 2.3724
Epoch 7/10
526/526 [==============================] - 274s 521ms/step - loss: 1.6964 - val_loss: 2.3193
Epoch 8/10
526/526 [==============================] - 270s 514ms/step - loss: 1.5004 - val_loss: 2.2826
Epoch 9/10
526/526 [==============================] - 270s 513ms/step - loss: 1.3338 - val_loss: 2.2714
Epoch 10/10
526/526 [==============================] - 270s 514m

## Step 6-3. 작문 모델 평가 - 작문 시켜보고 직접 평가

In [20]:
# 작문 모델 평가 - 작문 시켜보고 직접 평가
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love the way you lie <end> '

  
## Step 5-4. 인공지능 만들기: hyperparameter 변경

In [11]:
embedding_size = 256*2
hidden_size = 1024*2
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
# optimizer와 loss등은 차차 배웁니다

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=20, validation_data=(enc_val, dec_val))

Epoch 1/20
526/526 [==============================] - 340s 575ms/step - loss: 3.4597 - val_loss: 3.0629
Epoch 2/20
526/526 [==============================] - 308s 586ms/step - loss: 2.8963 - val_loss: 2.7926
Epoch 3/20
526/526 [==============================] - 308s 586ms/step - loss: 2.5955 - val_loss: 2.6158
Epoch 4/20
526/526 [==============================] - 309s 587ms/step - loss: 2.3119 - val_loss: 2.4833
Epoch 5/20
526/526 [==============================] - 309s 587ms/step - loss: 2.0380 - val_loss: 2.3863
Epoch 6/20
526/526 [==============================] - 309s 587ms/step - loss: 1.7823 - val_loss: 2.3149
Epoch 7/20
526/526 [==============================] - 309s 588ms/step - loss: 1.5520 - val_loss: 2.2686
Epoch 8/20
526/526 [==============================] - 349s 664ms/step - loss: 1.3551 - val_loss: 2.2455
Epoch 9/20
526/526 [==============================] - 310s 589ms/step - loss: 1.1959 - val_loss: 2.2459
Epoch 10/20
526/526 [==============================] - 309s 588m

## Step 6-4. 작문 모델 평가 - 작문 시켜보고 직접 평가

In [13]:
# 작문 모델 평가 - 작문 시켜보고 직접 평가
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you more than you ll ever know <end> '

  
## Step 5-5. 인공지능 만들기: hyperparameter 변경

In [14]:
embedding_size = 256*3
hidden_size = 1024*3
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [15]:
# optimizer와 loss등은 차차 배웁니다

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val))

Epoch 1/10
526/526 [==============================] - 674s 1s/step - loss: 3.4119 - val_loss: 3.0461
Epoch 2/10
526/526 [==============================] - 623s 1s/step - loss: 2.8944 - val_loss: 2.8035
Epoch 3/10
526/526 [==============================] - 623s 1s/step - loss: 2.6237 - val_loss: 2.6467
Epoch 4/10
526/526 [==============================] - 624s 1s/step - loss: 2.3709 - val_loss: 2.5262
Epoch 5/10
526/526 [==============================] - 624s 1s/step - loss: 2.1255 - val_loss: 2.4323
Epoch 6/10
526/526 [==============================] - 624s 1s/step - loss: 1.8901 - val_loss: 2.3601
Epoch 7/10
526/526 [==============================] - 624s 1s/step - loss: 1.6677 - val_loss: 2.3056
Epoch 8/10
526/526 [==============================] - 625s 1s/step - loss: 1.4652 - val_loss: 2.2738
Epoch 9/10
526/526 [==============================] - 624s 1s/step - loss: 1.2877 - val_loss: 2.2531
Epoch 10/10
526/526 [==============================] - 624s 1s/step - loss: 1.1422 - val_lo

## Step 6-5. 작문 모델 평가 - 작문 시켜보고 직접 평가

In [16]:
# 작문 모델 평가 - 작문 시켜보고 직접 평가
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you , i love you <end> '

  
## Step 5-6. 인공지능 만들기: hyperparameter 변경 (5-4와 같으며 epoch 조정)

In [17]:
embedding_size = 256*2
hidden_size = 1024*2
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [18]:
# optimizer와 loss등은 차차 배웁니다

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=8, validation_data=(enc_val, dec_val))

Epoch 1/8
526/526 [==============================] - 330s 592ms/step - loss: 3.4636 - val_loss: 3.0442
Epoch 2/8
526/526 [==============================] - 310s 590ms/step - loss: 2.8874 - val_loss: 2.7828
Epoch 3/8
526/526 [==============================] - 311s 591ms/step - loss: 2.5817 - val_loss: 2.5994
Epoch 4/8
526/526 [==============================] - 311s 591ms/step - loss: 2.2869 - val_loss: 2.4649
Epoch 5/8
526/526 [==============================] - 311s 591ms/step - loss: 2.0038 - val_loss: 2.3644
Epoch 6/8
526/526 [==============================] - 311s 591ms/step - loss: 1.7406 - val_loss: 2.2914
Epoch 7/8
526/526 [==============================] - 311s 592ms/step - loss: 1.5074 - val_loss: 2.2472
Epoch 8/8
526/526 [==============================] - 312s 592ms/step - loss: 1.3108 - val_loss: 2.2292


## Step 6-6. 작문 모델 평가 - 작문 시켜보고 직접 평가

In [19]:
# 작문 모델 평가 - 작문 시켜보고 직접 평가
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

# 4-7. 프로젝트: 멋진 작사가 만들기 
## Step 1. 데이터 다운로드
[문제]
- 이미 실습(1) 데이터 다듬기에서 Cloud shell에 심볼릭 링크로 ~/aiffel/lyricist/data를 생성하셨다면, ~/aiffel/lyricist/data/lyrics에 데이터가 있습니다.  

[실행]
- 실행

## Step 2. 데이터 읽어오기
[문제]
- glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요.
- glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 할게요!  

[실행]
- glob 함수를 처음 보는 것이어서 웹 검색하여 해결
- glob 함수를 써서 읽어온 텍스트 파일의 리스트 생성
- 여러 개의 파일을 읽어서 합쳐야 하므로 텍스트 파일 리스트에 대해 for문 사용
 
## Step 3. 데이터 정제
[문제]
- 앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!
- preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.
- 추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다.
  - 너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠.
  - 그래서 이번에는 문장을 토큰화 했을 때 **토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외**하기를 권합니다.  
  
[실행]
- 토큰 갯수가 15개 초과하는 문장을 제외할 때 "if len(sentence.split()) > 15: continue"를 사용
- sentence.split()를 못 찾아서 애먹었음 ㅎ~ 파이썬이 능숙치 못해 힘듬 ㅠㅠ

## Step 4. 평가 데이터셋 분리
[문제]
- 훈련 데이터와 평가 데이터를 분리하세요!
  - tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다.
- 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!  

[실행]
- 데이터 분리는 큰 문제 없이 해결
  - train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 42) 사용
- 단어장 크기는 tokenize() 함수의 num_words (=7000) 파라미터 값 변경: 7000 -> 12000

[문제]
- Source Train: (124960, 14)
- Target Train: (124960, 14)
  - 만약 결과가 다르다면 천천히 과정을 다시 살펴 동일한 결과를 얻도록 하세요!
  - 만약 학습 데이터 개수가 124960보다 크다면 위 Step 3.의 데이터 정제 과정을 다시 한번 검토해 보시기를 권합니다.  
  
[실행]
- Source Train: (134685, 14)
- Target Train: (134685, 14)
  - 교재의 결과와 다르게 나와 데이터 정제 과정을 여러 번 검토하였으나 특이사항 발견 못함... (pass^^)  


## Step 5. 인공지능 만들기
[문제] 
- 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 **val_loss 값을 2.2 수준으로** 줄일 수 있는 모델을 설계하세요! 
  - (Loss는 아래 제시된 Loss 함수를 그대로 사용!)
- 그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!  

[실행] 
- 아래와 같은 여섯 가지 경우를 실행:


|  Case | Embedding Size | Hidden Size | Epoch | val_loss | Remark | Lyrics generated |  
|-----|:-----:|:-----:|:-----:|:-----:|:----- |:----- |    
| 5-1 | 256 | 1024 | 10 | 2.5711 | Epoch 1->10까지 val_loss 지속 감소 | i love you |  
| 5-2 | 512 | 1024 | 10 | 2.5119 | Epoch 1->10까지 val_loss 지속 감소 | i love you, i love you |  
| 5-3 | 256 | 2048 | 10 | 2.2700 | Epoch 1->10까지 val_loss 지속 감소 | i love the way you lie |  
| 5-4 | 512 | 2048 | 20 | 2.4364 | Epoch 8에서 val_loss 최소값 2.2455 기록 후 다시 증가. loss 값은 계속 감소... overfitting | i love you more than you ll ever know |  
| 5-5 | 768 | 3072 | 10 | 2.2608 | Epoch 1->10까지 val_loss 지속 감소 | i love you, i love you, i love you |  
| **5-6** | **512** | **2048** | **8** | **2.2292** | Case 5-4의 Epoch=8 val_loss값이 2.2 수준의 최소이므로 5-4를, overfitting이 일어나지 않도록 Epoch수를 줄여서 재실행 | i love you |
  
- Embedding Size와 Hidden Size를 번갈아 증가시켜 가면서 val_loss값 변화 추이 관찰
- Embedding Size=512와 Hidden Size=2048(Case 5-4)까지는 이전 경우들보다 val_loss값 감소
  - 이 경우 Epoch 8 이후로 val_loss값이 증가하면서 Overfitting이 발생함
- Embedding Size=768과 Hidden Size=3072(Case 5-5)로 변화시켜 학습시킨 결과(val_loss 2.2608), 적은 값이긴 하나 Case 5-4 Epoch=8의 값(2.2455)보다 val_loss값 증가
- Case 5-4의 Epoch=8에서 val_loss값이 2.2 수준이므로 Epoch수를 줄여서 Case 5-6을 만듦
- Case 5-6을 학습시킨 결과 val_loss값이 2.2 수준(2.2292)이 되어 추가 모델 설계 종료
- 이 멋진 모델이 만들어낸 가사 한 줄은... 
###  i love you
  - 이 작사가는 미사려구보다는 직진인가요???

## Comment
- 교재에서 시키는 대로 따라하는 수준
- 파라미터의 변경도 주먹구구식
- 모델이 의미하는 바를 더 공부하고
  - 학습단계에 따른 accuracy/loss 변화 그래프도 그려보아야 하나 몰라서 못함
- 훈련데이터 분석도 보다 심도있게 해야
  - EDA를 제대로 해야 (문장별 단어 수 분포 등) 하나 몰라서 못함
- 작사가 수준의 AI 모델을 개발할 수 있을 것으로 기대됨
- (추가로...) 모델 학습시간이 많이(?) 소요되고 초보자의 입장에서는 배우는 양이 많아서 Exploration 과제에 많은 시간 투자도 어려움
  - 시간을 더 투자하고 지식이 늘어난다면 더 잘 할 수 있을 듯...